# STAT 301 Group 40 Final Report

Wasay Hayat, Juhi Grover, Qi Xu, Allen Zheng

## Introduction

(add background information)

**Overarching question:** (insert question)

### Data Description

## Methods and Results

### Exploratory Data Analysis

### Plan

### Computational Code and Output

## Discussion

## References

(at least two references expected in the introduction)

- reference 1
- reference 2